In [1]:
# import dependencies

from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers



  Cloning https://github.com/deepset-ai/FARM.git (to revision 0749dcb8fb46dace0d0987d6fdedf5f28120a461) to /private/var/folders/rv/zk56y5v50qvgw72s5gdl57wh0000gn/T/pip-req-build-zhiikbmb
  Running command git clone -q https://github.com/deepset-ai/FARM.git /private/var/folders/rv/zk56y5v50qvgw72s5gdl57wh0000gn/T/pip-req-build-zhiikbmb
  Running command git rev-parse -q --verify 'sha^0749dcb8fb46dace0d0987d6fdedf5f28120a461'
  Running command git fetch -q https://github.com/deepset-ai/FARM.git 0749dcb8fb46dace0d0987d6fdedf5f28120a461
  Running command git checkout -q 0749dcb8fb46dace0d0987d6fdedf5f28120a461
     |████████████████████████████████| 119.5 MB 36 kB/s s eta 0:00:01   |████▏                           | 15.7 MB 3.8 MB/s eta 0:00:28██████████▊             | 69.7 MB 7.3 MB/s eta 0:00:07     |███████████████████▊            | 73.8 MB 7.3 MB/s eta 0:00:07     |███████████████████████████▌    | 102.5 MB 4.5 MB/s eta 0:00:04
     |████████████████████████████████| 74 kB 5.4 MB/s  et

  Using cached s3transfer-0.3.4-py2.py3-none-any.whl (69 kB)
     |████████████████████████████████| 52 kB 4.9 MB/s  eta 0:00:01
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached scikit_learn-0.24.1-cp37-cp37m-macosx_10_13_x86_64.whl (7.2 MB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Created wheel for farm: filename=farm-0.4.2-py3-none-any.whl size=132406 sha256=9cd17e9512887669d00ecce17df58796d7f3f3f472569317ca3f84f19a16111b
  Stored in directory: /Users/sava/Library/Caches/pip/wheels/ab/c6/07/8a6d54e8a1e32ffd4bde9e3b60194b023f6b2dc6fd09c16988
  Created wheel for databricks-cli: filename=databricks_cli-0.14.2-py3-none-any.whl size=100732 sha256=486cf03d1aaacbe2a7bf1bf929e875c2a674443bc6ced18c8d70936866a1581e
  Stored in directory: /Users/sava/Library/Caches/pip/wheels/54/8a/cd/8861fff0bea4a752ffe7561eb74e130c9ae38cdb7e43eb7b66
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-n

     |████████████████████████████████| 475 kB 22.8 MB/s eta 0:00:01
     |████████████████████████████████| 869 kB 18.4 MB/s eta 0:00:01


     |████████████████████████████████| 2.6 MB 32.0 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 7.1 MB/s  eta 0:00:01
  Created wheel for farm-haystack: filename=farm_haystack-0.1.3-py3-none-any.whl size=29907 sha256=51c8a31958997ae244b98e4b632d0170b87c7c1b41336420e18afe8fa9ce4116
  Stored in directory: /Users/sava/Library/Caches/pip/wheels/12/24/1a/924f8b6d170c42af7e8fa4ccf91cb3a1e55b7bd0e947e1d50b
  Created wheel for farm: filename=farm-0.4.1-py3-none-any.whl size=127341 sha256=dba1ce96de17d5d1e388783bfdef81fc82bed3983cfc315ad2565b1ac0ad3e69
  Stored in directory: /Users/sava/Library/Caches/pip/wheels/92/64/bc/11ed4631612f8e410699829d9fa6bc9347db92a17ffe8e5f8a
Successfully built farm-haystack farm
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.5.2
    Uninstalling tokenizers-0.5.2:
      Successfully uninstalled tokenizers-0.5.2
  Attempting uninstall: transformers

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [9]:
# connect to ES, create indexes
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

# to delete created indexes later on use:
# curl -XDELETE localhost:9200/label
# curl -XDELETE localhost:9200/document


03/11/2021 20:17:11 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.957s]
03/11/2021 20:17:12 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.422s]


In [10]:
# add data to ES

from lxml import etree
from lxml.etree import tostring

input_file_path = "data/Wikipedia-Strength-Training.xml"


tree = etree.parse(input_file_path)
root = tree.getroot()
# iterate through all the titles

document_dictionaries = []
for text_node in root.findall(".//text", namespaces=root.nsmap)[:20]:
    document_dictionaries.append({'text': text_node.text, 'meta': None})

document_store.write_documents(document_dictionaries)    


03/11/2021 20:17:20 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.859s]


In [11]:
# initialize retriever
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)


In [12]:
# initialize reader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


03/11/2021 20:17:30 - INFO - farm.utils -   Using device: CPU 
03/11/2021 20:17:30 - INFO - farm.utils -   Number of GPUs: 0
03/11/2021 20:17:30 - INFO - farm.utils -   Distributed Training: False
03/11/2021 20:17:30 - INFO - farm.utils -   Automatic Mixed Precision: None
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
03/11/2021 20:17:46 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
03/11/2021 20:17:46 - INFO - farm.utils -   Using device: CPU 
03/11/2021 20:17:46 - INFO - farm.utils -   Number of GPUs: 0
03/11/2021 20:17:46 - INFO - farm.utils -   Distributed Training: False
03/11/2021 20:17:46 - INFO - farm.utils -   Automatic Mixed Precision: None
03/11/2021 20

In [13]:
# create pipeline

from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [14]:
# make predictions
prediction1 = pipe.run(query="What is stamina?", top_k_retriever=10, top_k_reader=5)

prediction2 = pipe.run(query="How to avoid injury?", top_k_retriever=10, top_k_reader=5)


03/11/2021 20:18:06 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.055s]
Inferencing Samples: 100%|██████████| 1/1 [00:03<00:00,  3.02s/ Batches]
03/11/2021 20:23:11 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.026s]
Inferencing Samples: 100%|██████████| 2/2 [00:41<00:00, 20.56s/ Batches]


In [16]:
# get answers
print("Question 'What is stamina?'")
print_answers(prediction1, details="minimal")
print("Question 'How to avoid injury?'")
print_answers(prediction2, details="minimal")



Question 'What is stamina?'
[   {   'answer': 'personality trait',
        'context': '2-51, Washington DC.</ref>\n'
                   '\n'
                   'Endurance may also refer to an [[Grit (personality '
                   'trait)|ability to persevere through a difficult '
                   'situation]].\n'
                   '\n'
                   '== Traini'},
    {   'answer': 'the ability of an [[organism]] to exert itself and remain '
                  'active for a long period of time',
        'context': 'sychological)|hardiness]]) is the ability of an '
                   '[[organism]] to exert itself and remain active for a long '
                   'period of time, as well as its ability to re'},
    {   'answer': 'strength',
        'context': 'robic]] or [[anaerobic exercise|anaerobic]] ability, i.e. '
                   'endurance or strength. A well-rounded fitness program '
                   'improves a person in all aspects of fi'},
    {   'answer': 'effec